
Las tablas son generadas en **Modo Batch**, y se almacenan como tablas Delta en el Data Lake, siguiendo el modelo Medallón.

## Ejecución

- Todas las tablas se generan de forma **Batch**, utilizando los datos enriquecidos de la capa Silver.
- Los resultados se almacenan como **Tablas Delta** listas para ser consultadas o visualizadas.
- Pueden ser ejecutadas manualmente desde notebooks o programadas, en data factory se opto por la utilizacion de **Schedule Trigger**

---


### 1. Ventas por Día, Canal y Región
- 📌 **Descripción**: Agrega la información de ventas diarias segmentadas por canal y región.
- 📈 **Aplicación**: Monitoreo de rendimiento por zonas y análisis temporal de ventas.


In [0]:
from pyspark.sql.functions import *
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")


sales_by_day = (
    df_gold
    .groupBy("event_date", "channel_name", "store_region")
    .agg(
        count("*").alias("total_transactions"),
        sum("amount").alias("total_sales"),
        avg("amount").alias("avg_ticket")
    )
)

sales_by_day.write.mode("overwrite").format("delta").save("/mnt/gold/sales/sales_by_day")
display(sales_by_day)


event_date,channel_name,store_region,total_transactions,total_sales,avg_ticket
2025-07-20,Caja Automática,Región AMBA,26,6387.6,245.6769230769231
2025-07-16,Caja Automática,Región AMBA,2,802.7,401.35
2025-07-17,Caja Automática,Región AMBA,1,91.17,91.17
2025-07-15,Caja Automática,Región Norte,1,190.52,190.52
2025-07-20,Caja Automática,Región Sur,13,2917.9900000000002,224.46076923076924
2025-07-17,Caja Automática,Región Sur,1,49.78,49.78
2025-07-18,Caja Automática,Región Sur,2,432.13,216.065
2025-07-18,Ecommerce,Región AMBA,2,230.0,115.0
2025-07-18,Caja Automática,Región AMBA,1,53.56,53.56
2025-07-20,Caja Automática,Región Norte,15,3169.92,211.328


### 2. Top Clientes
- 📌 **Descripción**: Identifica los clientes con mayor nivel de gasto total.
- 👥 **Aplicación**: Segmentación de clientes, programas de fidelidad, y campañas de marketing.

In [0]:

df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

top_customers = (
    df_gold
    .groupBy("customer_id", "customer_name", "customer_loyalty_tier")
    .agg(
        sum("amount").alias("total_spent"),
        count("*").alias("transactions"),
        avg("amount").alias("avg_ticket"),
        max("event_date").alias("last_purchase_date")
    )
    .orderBy(desc("total_spent"))
)

display(top_customers)
top_customers.write.mode("overwrite").format("delta").save("/mnt/gold/sales/top_customers")


customer_id,customer_name,customer_loyalty_tier,total_spent,transactions,avg_ticket,last_purchase_date
CUST002,Carlos Pérez,Silver,10307.77,38,271.2571052631579,2025-07-20
CUST003,Ana Ruiz,Bronze,9606.84,36,266.8566666666667,2025-07-20
CUST001,María Gómez,Gold,9200.88,37,248.6724324324324,2025-07-20
CUST005,Laura Acosta,Silver,8661.05,36,240.5847222222222,2025-07-20
CUST004,Juan Torres,Gold,8466.880000000001,35,241.91085714285717,2025-07-20


### 3. Rendimiento de Productos
- 📌 **Descripción**: Evalúa el desempeño de los productos en términos de ventas e ingresos.
- 📦 **Aplicación**: Gestión de inventario, decisiones de pricing y desarrollo de productos.


In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

product_performance = (
    df_gold
    .groupBy("product_id", "product_name")
    .agg(
        count("*").alias("units_sold"),
        sum("amount").alias("total_revenue"),
        avg("amount").alias("avg_ticket")
    )
    .orderBy(desc("total_revenue"))
)

display(product_performance)
product_performance.write.mode("overwrite").format("delta").save("/mnt/gold/sales/product_performance")


product_id,product_name,units_sold,total_revenue,avg_ticket
P002,Pan integral,30,7987.73,266.25766666666664
P003,Queso cremoso,27,6666.700000000001,246.91481481481483
P001,Leche entera,19,5164.799999999999,271.83157894736837
P005,Yerba mate,22,4984.300000000001,226.55909090909097
P004,Coca Cola 1.5L,19,4000.0599999999995,210.5294736842105



### 4. Ventas por Región y Canal
- 📌 **Descripción**: Muestra un cruce entre regiones de venta y canales utilizados.
- 🗺️ **Aplicación**: Análisis geográfico y por tipo de canal, útil para visualizaciones tipo mapa de calor o matrices comparativas.

In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

sales_by_region_channel = (
    df_gold
    .groupBy("store_region", "channel_name")
    .agg(
        sum("amount").alias("total_sales"),
        count("*").alias("transactions")
    )
)

display(sales_by_region_channel)
sales_by_region_channel.write.mode("overwrite").format("delta").save("/mnt/gold/sales/sales_by_region_channel")


store_region,channel_name,total_sales,transactions
Región Sur,Caja Automática,3830.8699999999994,18
Región Norte,Caja Automática,3688.8900000000003,18
Región AMBA,Caja Automática,7335.030000000001,30
Región AMBA,Ecommerce,4668.72,22
Región AMBA,Punto de Venta Físico,7714.88,24
Región Norte,Punto de Venta Físico,3642.66,16
Región Sur,Punto de Venta Físico,4749.21,18
Región Norte,Ecommerce,5114.29,19
Región Sur,Ecommerce,5498.87,17


### 5. Distribución Diaria por Categoría de Ticket
- 📌 **Descripción**: Clasifica las ventas diarias según el rango de importe del ticket.
- 🎯 **Aplicación**: Evaluación de hábitos de consumo y detección de tendencias de compra.


In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

daily_ticket_distribution = (
    df_gold
    .groupBy("event_date", "ticket_category")
    .agg(
        count("*").alias("transaction_count"),
        sum("amount").alias("total_amount")
    )
    .orderBy("event_date", "ticket_category")
)

display(daily_ticket_distribution)
daily_ticket_distribution.write.mode("overwrite").format("delta").save("/mnt/gold/sales/daily_ticket_distribution")


event_date,ticket_category,transaction_count,total_amount
2025-07-15,ALTO,4,1444.8399999999997
2025-07-15,BAJO,2,41.84
2025-07-15,MEDIO,1,190.52
2025-07-16,ALTO,8,2533.51
2025-07-16,BAJO,1,10.22
2025-07-16,MEDIO,6,807.7300000000001
2025-07-17,ALTO,8,3023.4300000000003
2025-07-17,BAJO,1,49.78
2025-07-17,MEDIO,3,295.93
2025-07-18,ALTO,2,703.85


### 6. Rendimiento por Sucursal (Extra)
- 📌 **Descripción**: Mide el desempeño de cada sucursal en base a sus ventas y frecuencia de transacciones.
- 🏬 **Aplicación**: Comparativa entre locales, decisiones de expansión, análisis por región.


In [0]:
df_gold = spark.read.format("delta").load("/mnt/silver/sales/enriched")

store_performance = (
    df_gold
    .groupBy("store", "store_name", "store_region")
    .agg(
        count("*").alias("transactions"),
        sum("amount").alias("total_sales"),
        avg("amount").alias("avg_ticket")
    )
    .orderBy(desc("total_sales"))
)

display(store_performance)
store_performance.write.mode("overwrite").format("delta").save("/mnt/gold/sales/store_performance")


store,store_name,store_region,transactions,total_sales,avg_ticket
Sucursal A,Tienda Central,Región AMBA,76,19718.63,259.4556578947369
Sucursal C,Tienda Sur,Región Sur,53,14078.949999999999,265.6405660377358
Sucursal B,Tienda Norte,Región Norte,53,12445.84,234.82716981132074


## Creación de tablas en hive metastore

In [0]:
spark.sql(
    """
        CREATE SCHEMA IF NOT EXISTS capa_gold;
    """
)
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.sales_by_day
     USING DELTA
    LOCATION '/mnt/gold/sales/sales_by_day';
    """
)

DataFrame[]

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.top_customers
USING DELTA
LOCATION '/mnt/gold/sales/top_customers';
    """
)


DataFrame[]

In [0]:
spark.sql(
    """
    CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.product_performance
USING DELTA
LOCATION '/mnt/gold/sales/product_performance';
    """
)

DataFrame[]

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.sales_by_region_channel
USING DELTA
LOCATION '/mnt/gold/sales/sales_by_region_channel';
    """
)

DataFrame[]

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.daily_ticket_distribution
USING DELTA
LOCATION '/mnt/gold/sales/daily_ticket_distribution';
    """
)

DataFrame[]

In [0]:
spark.sql(
    """
        CREATE TABLE IF NOT EXISTS hive_metastore.capa_gold.store_performance
USING DELTA
LOCATION '/mnt/gold/sales/store_performance';
    """
)

DataFrame[]